In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from src.dset import DatasetPhysNetED
from src.archs import PhysNetED

import numpy as np
import configparser
import os
import json
import cv2
import matplotlib.pyplot as plt
from skimage import data, io, filters
from tqdm import tqdm

def train_loop(trainloader, model, loss_fn, optimizer):
    model.train()
    size = len(trainloader.dataset)
    train_preds = np.empty(0)
    train_targets = np.empty(0)

    for batch, (inputs, targets) in enumerate(trainloader):
        inputs = inputs.to(device)
        targets = targets.to(device).squeeze()
        pred = model(inputs).squeeze()

        for cnt, pr in enumerate(pred):
            if (batch_size*batch+cnt)%(int(1/overlap)) == 0:
                p = pr.cpu().detach().numpy().squeeze().flatten()
                t = targets[cnt].cpu().detach().numpy().squeeze().flatten()
                train_preds = np.concatenate((train_preds,p))
                train_targets = np.concatenate((train_targets,t))
                
        loss = loss_fn(pred, targets)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1 == 0:
            loss, current = loss.item(), batch * len(targets)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    return train_preds, train_targets
    
def test_loop(testloader, model, loss_fn):
    size = len(testloader.dataset)
    model.eval()    
    test_loss, correct = 0, 0
    test_preds = np.empty(0)
    test_targets = np.empty(0)

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs = inputs.to(device)
            pred = model(inputs).squeeze()
            
            p = pred.cpu().detach().numpy().squeeze().flatten()
            t = targets.cpu().detach().numpy().squeeze().flatten()
            test_preds = np.concatenate((test_preds,p))
            test_targets = np.concatenate((test_targets,t))
        
    return test_preds, test_targets

# if __name__ == '__main__':

configFilename = './config/PURE_PhysNet.cfg'

parser = configparser.ConfigParser(inline_comment_prefixes=('#', ';'))
parser.optionxform = str
parser.read(configFilename)

cfgdict = dict(parser['DEFAULT'].items())
vdir, sigdir = cfgdict['videodataDIR'], cfgdict['signalpath']
print(vdir, sigdir, cfgdict)

# overlap = int(cfgdict['overlap'])
# batch_size = int(cfgdict['batch_size'])
overlap = 0.75
batch_size = 32

print(f'overlap: {overlap}')
print(f'batch_size: {batch_size}')

trainset = DatasetPhysNetED(cfgdict, start = 0, end = 1600, overlap = overlap)
testset = DatasetPhysNetED(cfgdict, start = 1601, end = 2009, overlap = 0)

# Construct DataLoaders
trainloader = DataLoader(trainset,
                        batch_size = batch_size)
testloader = DataLoader(testset,
                        batch_size = batch_size)

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Load Model
model = PhysNetED()
if device != 'cpu':
    model = model.cuda()
loss_fn = nn.MSELoss()
learning_rate = float(cfgdict['learning_rate'])
optimizer =  torch.optim.AdamW(model.parameters(), lr=learning_rate)


# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter('logs/fashion_mnist_experiment_1')

model_name = 'PhysNet_v8'

if not os.path.exists('checkpoints'):
    os.mkdir('checkpoints')
if not os.path.exists('checkpoints/'+model_name):
    os.mkdir('checkpoints/'+model_name)


epochs = 100
train_losses = []
test_losses = []


/home/sim/Dataset/rPPG/PURE/01-01/01-01 /home/sim/Dataset/rPPG/PURE/01-01/01-01.json {'dataset': 'PURE', 'model': 'PhysNet', 'videodataDIR': '/home/sim/Dataset/rPPG/PURE/01-01/01-01', 'signalpath': '/home/sim/Dataset/rPPG/PURE/01-01/01-01.json', 'learning_rate': '1e-3', 'depth': '32', 'height': '128', 'width': '128', 'crop': 'True', 'overlap': '0.75', 'batch_size': '32'}
overlap: 0.75
batch_size: 32
Using cuda device


## 레이어 이름 불러오기 (예, ResNet 18)

In [2]:
from torchvision import models
rn18 = models.resnet18(pretrained=True)

children_counter = 0
for n,c in rn18.named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n,)
    children_counter+=1

Children Counter:  0  Layer Name:  conv1
Children Counter:  1  Layer Name:  bn1
Children Counter:  2  Layer Name:  relu
Children Counter:  3  Layer Name:  maxpool
Children Counter:  4  Layer Name:  layer1
Children Counter:  5  Layer Name:  layer2
Children Counter:  6  Layer Name:  layer3
Children Counter:  7  Layer Name:  layer4
Children Counter:  8  Layer Name:  avgpool
Children Counter:  9  Layer Name:  fc


In [3]:
c

Linear(in_features=512, out_features=1000, bias=True)

## Layer 모듈을 Ordered Dictionary로 받아오기

In [4]:
rn18._modules

OrderedDict([('conv1',
              Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)),
             ('bn1',
              BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
             ('relu', ReLU(inplace=True)),
             ('maxpool',
              MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)),
             ('layer1',
              Sequential(
                (0): BasicBlock(
                  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (relu): ReLU(inplace=True)
                  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                )
                (1): BasicBlock(
         

# PhysNet으로 중간 Layer Output 받오기 실습

## 하위 모듈의 이름과 내용을 받아오는 방법

## 1. nn.module의 하위 모듈 (sub modules)을 iterator로 불러오기

In [113]:
model = PhysNetED()
if device != 'cpu':
    model = model.cuda()
    
for cnt, m in enumerate(model.modules()):
    print(f'*** {cnt} : {m}')

*** 0 : PhysNetED(
  (start): Sequential(
    (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
  )
  (loop1): Sequential(
    (0): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
    (1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
  )
  (encoder): Sequential(
    (0): AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0)
    (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

## 2. 하위 모듈의 이름과 내용을 iterator로 받아오기

In [96]:
for cnt, m in enumerate(model.named_modules()):
    print(f'*** {cnt} : {m}')

*** 0 : ('', PhysNetED(
  (start): Sequential(
    (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
  )
  (loop1): Sequential(
    (0): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
    (1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
  )
  (encoder): Sequential(
    (0): AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0)
    (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

## 3. 하위 모듈의 내용과 이름을 불러오는 다른 방법

In [104]:
for cnt, (name, content) in enumerate(model.named_children()):
    print("Children Counter: ", cnt," Layer Name: ",name, ' *** Content: ', content)

Children Counter:  0  Layer Name:  start  *** Content:  Sequential(
  (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
  (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ELU(alpha=1.0)
)
Children Counter:  1  Layer Name:  loop1  *** Content:  Sequential(
  (0): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
  (1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ELU(alpha=1.0)
  (4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ELU(alpha=1.0)
)
Children Counter:  2  Layer Name:  encoder  *** Content:  Sequential(
  (0): AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0)
  (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)

## 4. 하위 모듈의 이름을 불러오는 다른 방법

In [108]:
model._modules.keys()

odict_keys(['start', 'loop1', 'encoder', 'loop4'])

## 5. 하위 모듈 중 하나 (sequential 모듈)의 하위 모듈을 불러오는 방법

In [109]:
for n,c in model._modules['loop1'].named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n, c)
    children_counter+=1

Children Counter:  17  Layer Name:  0 AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
Children Counter:  18  Layer Name:  1 Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
Children Counter:  19  Layer Name:  2 BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Children Counter:  20  Layer Name:  3 ELU(alpha=1.0)
Children Counter:  21  Layer Name:  4 Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
Children Counter:  22  Layer Name:  5 BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Children Counter:  23  Layer Name:  6 ELU(alpha=1.0)


## 하위 모듈을 변형하는 방법

In [137]:
model = PhysNetED()
if device != 'cpu':
    model = model.cuda()
    
list(model._modules.keys())

['start', 'loop1', 'encoder', 'loop4', 'decoder', 'end']

In [115]:
model._modules.pop('end')

Sequential(
  (0): AdaptiveAvgPool3d(output_size=(None, 1, 1))
  (1): Conv3d(64, 1, kernel_size=(1, 1, 1), stride=(1, 1, 1))
)

In [116]:
model

PhysNetED(
  (start): Sequential(
    (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
  )
  (loop1): Sequential(
    (0): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
    (1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
  )
  (encoder): Sequential(
    (0): AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0)
    (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (

In [13]:
model._modules.pop('decoder')

Sequential(
  (0): ConvTranspose3d(64, 64, kernel_size=(4, 1, 1), stride=(2, 1, 1), padding=(1, 0, 0))
  (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ELU(alpha=1.0)
  (3): ConvTranspose3d(64, 64, kernel_size=(4, 1, 1), stride=(2, 1, 1), padding=(1, 0, 0))
  (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ELU(alpha=1.0)
)

## 모델의 weight를 불러오는 방법

In [122]:
model.start[0].weight

Parameter containing:
tensor([[[[[ 0.0963, -0.0833,  0.0396,  0.0071,  0.0276],
           [ 0.0028, -0.0728, -0.0995,  0.0688,  0.0025],
           [-0.0203, -0.0626,  0.0704, -0.0712,  0.0240],
           [ 0.0330, -0.0517, -0.1059,  0.0559,  0.0344],
           [ 0.0634, -0.0390,  0.1151,  0.0638,  0.1108]]],


         [[[-0.0811, -0.1099, -0.0520,  0.0273, -0.1065],
           [-0.0343, -0.0977,  0.0440,  0.0144,  0.0870],
           [-0.0545, -0.0484, -0.0416,  0.0855,  0.0412],
           [-0.0410, -0.0022, -0.0872,  0.0916, -0.0339],
           [ 0.0583, -0.0247, -0.0411,  0.0390,  0.0417]]],


         [[[-0.0456,  0.0529,  0.0801,  0.0196, -0.0715],
           [-0.1049, -0.0038, -0.0465, -0.0685, -0.0363],
           [ 0.0305, -0.0641, -0.0994, -0.0187, -0.0215],
           [ 0.0274,  0.0895,  0.0184, -0.0872,  0.0168],
           [ 0.0624,  0.0205, -0.0367, -0.0601, -0.0524]]]],



        [[[[-0.0729,  0.0866,  0.0674,  0.0424, -0.0425],
           [ 0.0489,  0.0174, -0.077

In [140]:
pretrained.start.parameters

<bound method Module.parameters of Sequential(
  (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
  (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ELU(alpha=1.0)
)>

## How to get intermediate layer output in a model

In [134]:
model = PhysNetED()
if device != 'cpu':
    model = model.cuda()
    
class new_model(nn.Module):
    def __init__(self, model, output_layer = None):
        super().__init__()
        layers = list(model._modules.keys())
        layer_count = 0
        for l in layers:
            if l != output_layer:
                layer_count += 1
            else:
                break
        for i in range(1,len(layers)-layer_count):
            dummy_var = model._modules.pop(layers[-i])
        self.net = nn.Sequential(model._modules)

    def forward(self,x):
        x = self.net(x)
        return x
    
model2 = new_model(model, output_layer = 'loop4')
model2

new_model(
  (net): Sequential(
    (start): Sequential(
      (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
    )
    (loop1): Sequential(
      (0): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
      (1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ELU(alpha=1.0)
      (4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ELU(alpha=1.0)
    )
    (encoder): Sequential(
      (0): AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0)
      (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (2): BatchNorm3d(64, eps=1e-05, m

In [135]:
model

PhysNetED(
  (start): Sequential(
    (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
  )
  (loop1): Sequential(
    (0): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
    (1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
  )
  (encoder): Sequential(
    (0): AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0)
    (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (

In [ ]:
video, target = trainset.__getitem__(0)

In [22]:
video.shape, target.shape

(torch.Size([3, 32, 128, 128]), torch.Size([32]))

In [ ]:
for videos, targets in trainloader:
    break

In [28]:
videos[:2].shape

torch.Size([2, 3, 32, 128, 128])

## Cuda 디바이스 선택하는 방법

In [31]:
videos.device

device(type='cpu')

In [32]:
device

'cuda'

In [40]:
videos = videos.to(device)

In [42]:
videos.device

device(type='cuda', index=0)

In [47]:
 torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.get_device_name(device)

(True, 2, 0, 'Quadro RTX 8000')

In [141]:
!nvidia-smi

Wed Jun  2 15:51:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:1A:00.0 Off |                  Off |
| 33%   30C    P8    10W / 260W |   4519MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1D:00.0 Off |                  N/A |
| 31%   

In [48]:
# GPU 할당 변경하기
GPU_NUM = 1 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=1)

In [49]:
torch.cuda.set_device(device) # change allocation of current GPU

## 그냥 pop 시킨 모델은 여전히 decoder와 end 모듈을 불러온다.

In [50]:
model(videos[:2])

AttributeError: 'PhysNetED' object has no attribute 'decoder'

### 그냥 sequential로 묶고 class로 foward까지 정의해주지 않으면 에러남

In [142]:
model3 = nn.Sequential(model._modules)
output = model3(videos[:2])
output.shape

RuntimeError: Expected tensor for argument #1 'input' to have the same device as tensor for argument #2 'weight'; but device 0 does not equal 1 (while checking arguments for cudnn_convolution)

In [59]:
output = model2(videos[:2])
output.shape

torch.Size([2, 64, 8, 8, 8])

In [61]:
targets.shape

torch.Size([32, 32])

## 새로 intermediate까지만 잘라서 만든 모델과 원래 모델의 weight가 같음을 비교

In [92]:
model2.net.start[0].weight

Parameter containing:
tensor([[[[[ 1.1170e-01, -1.0185e-01, -3.7801e-02, -3.0368e-02,  9.9623e-02],
           [ 7.9924e-02,  8.7121e-02, -4.5142e-02, -6.7819e-03, -1.1263e-01],
           [-4.3391e-02, -4.3082e-02, -1.1236e-01,  7.7422e-03, -1.0851e-01],
           [ 8.4645e-02,  3.4432e-02,  5.7707e-02, -2.7472e-02, -1.0305e-01],
           [ 5.3895e-02,  1.1404e-01, -9.0173e-02, -5.9601e-02, -4.9683e-03]]],


         [[[-1.5307e-02,  4.9154e-02, -7.5848e-03,  8.8282e-02, -4.5205e-02],
           [ 3.1297e-02, -3.3680e-02, -1.0099e-01, -5.7649e-02, -5.3909e-02],
           [ 5.2862e-02,  4.4811e-02,  2.9655e-02,  6.5812e-02, -1.0326e-01],
           [ 9.0073e-02,  1.0810e-01, -1.1537e-01,  6.6733e-02,  8.4546e-03],
           [-7.9090e-02, -5.4057e-02,  7.4986e-02, -5.6194e-02, -8.8540e-02]]],


         [[[ 4.0820e-02,  1.1078e-01,  9.3464e-02, -9.0325e-02, -6.1319e-02],
           [-8.2835e-03, -7.2220e-03, -7.9017e-02, -1.8941e-03,  1.1214e-01],
           [ 1.1356e-01,  1.0963e-

In [93]:
pretrained.start[0].weight

Parameter containing:
tensor([[[[[ 1.1170e-01, -1.0185e-01, -3.7801e-02, -3.0368e-02,  9.9623e-02],
           [ 7.9924e-02,  8.7121e-02, -4.5142e-02, -6.7819e-03, -1.1263e-01],
           [-4.3391e-02, -4.3082e-02, -1.1236e-01,  7.7422e-03, -1.0851e-01],
           [ 8.4645e-02,  3.4432e-02,  5.7707e-02, -2.7472e-02, -1.0305e-01],
           [ 5.3895e-02,  1.1404e-01, -9.0173e-02, -5.9601e-02, -4.9683e-03]]],


         [[[-1.5307e-02,  4.9154e-02, -7.5848e-03,  8.8282e-02, -4.5205e-02],
           [ 3.1297e-02, -3.3680e-02, -1.0099e-01, -5.7649e-02, -5.3909e-02],
           [ 5.2862e-02,  4.4811e-02,  2.9655e-02,  6.5812e-02, -1.0326e-01],
           [ 9.0073e-02,  1.0810e-01, -1.1537e-01,  6.6733e-02,  8.4546e-03],
           [-7.9090e-02, -5.4057e-02,  7.4986e-02, -5.6194e-02, -8.8540e-02]]],


         [[[ 4.0820e-02,  1.1078e-01,  9.3464e-02, -9.0325e-02, -6.1319e-02],
           [-8.2835e-03, -7.2220e-03, -7.9017e-02, -1.8941e-03,  1.1214e-01],
           [ 1.1356e-01,  1.0963e-

## 아래 코드의 출처
https://medium.com/the-owl/extracting-features-from-an-intermediate-layer-of-a-pretrained-model-in-pytorch-c00589bda32b

In [ ]:
class new_model(nn.Module):
    def __init__(self,output_layer = None):
        super().__init__()
        self.pretrained = model
        self.output_layer = output_layer
        self.layers = list(self.pretrained._modules.keys())
        self.layer_count = 0
        for l in self.layers:
            if l != self.output_layer:
                self.layer_count += 1
            else:
                break
        for i in range(1,len(self.layers)-self.layer_count):
            self.dummy_var = self.pretrained._modules.pop(self.layers[-i])
        
        self.net = nn.Sequential(self.pretrained._modules)
        self.pretrained = None

    def forward(self,x):
        x = self.net(x)
        return x